In [1]:
import glob
import numpy as np
import pandas as pd
from zipfile import ZipFile
import bz2
import h5netcdf
import xarray as xr
import netCDF4 as nc


In [2]:
import ray
# import time
ray.init(num_cpus=100)

2023-07-03 11:01:26,856	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.1.0


In [79]:
# ! pip install h5netcdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.6 MB/s eta 0:00:00


## old files

In [3]:
glob.glob('/users/staff/leo/fastscratch/rise/1.0/exp01/*/rharm_h_??????.nc')

['/users/staff/leo/fastscratch/rise/1.0/exp01/072215/rharm_h_072215.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/011520/rharm_h_011520.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/008023/rharm_h_008023.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/048650/rharm_h_048650.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/028661/rharm_h_028661.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/070273/rharm_h_070273.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/003953/rharm_h_003953.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/044292/rharm_h_044292.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/061024/rharm_h_061024.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/008495/rharm_h_008495.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/091413/rharm_h_091413.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/070414/rharm_h_070414.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/074455/rharm_h_074455.nc',
 '/users/staff/leo/fastscratch/rise/1.0/exp01/06068

## unzip and fix errors from input files -> write to csv

In [124]:
@ray.remote
def unzip_and_fix(rh_i):
    try:
        out_dir = '/users/staff/uvoggenberger/scratch/RHARM_2/'+rh_i.split('/')[-1].split('.bz2')[0]
        with bz2.BZ2File(rh_i) as bz_file:
            line=bz_file.readlines()
            out = [line[0].decode()]
            for i_line in line[1:]:
                x_line = i_line.decode().replace(' ', '')
                x_line = ''.join([x_line.split('"')[0], x_line.split('"')[2]])
                if x_line.count(',') > 100:
                    print('Too many vars: ', rh_i, x_line.count(','), x_line)
                out.append(x_line)
        with open(out_dir,'wb') as file:
            for line in out:
                to_write = line.encode()
                file.write(to_write)
                if to_write[-1:] != b'\n':
                    file.write(b'\n')
        return 0
    
    except:
        return 1, file
# failed


In [125]:
result_ids = []
for file in glob.glob('/users/staff/uvoggenberger/scratch/RHARM_2/*.bz2')[:]:
    result_ids.append(unzip_and_fix.remote(file))

results = ray.get(result_ids)
ray.shutdown()
results

2023-06-29 15:54:08,108	INFO worker.py:1528 -- Started a local Ray instance.


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


## write to netcdf

In [126]:
@ray.remote
def write_to_nc(file):
    try:
        df = pd.read_csv(file, low_memory=False)
        renamer = {}
        for i in df.keys():
            renamer[i] = i.split(' ')[-1]
        df = df.rename(columns=renamer)
        df_sel = df[['rh', 'u', 'v', 'press', 'temp', 'reltime', 'latitude', 'longitude']].copy()
        df_sel['reltime'] = pd.to_datetime(df_sel['reltime'], format='%Y-%m-%d%H:%M:%S+%f')

        splevs=[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
        df_sel = df_sel.copy()[np.isin(df_sel['press'], splevs)]

        avail = list(df_sel.press.drop_duplicates())
        for i in splevs:
            if i in avail:
                pass
            else:
                # print(i)
                df_sel.loc[len(df_sel.index)] = [np.nan, np.nan, np.nan, i, np.nan, df_sel.reltime.iloc[0], df_sel.latitude.iloc[0], df_sel.longitude.iloc[0]]

        df_sel =  df_sel.copy()
        df_sel['hour'] = df_sel['reltime'].dt.hour
        hour_binary = np.array([0]*len(df_sel['hour']))
        hour_binary[np.logical_and(df_sel['hour'] > 6 , df_sel['hour'] <= 18)] = 1
        df_sel['hour'] = hour_binary

        df_sel['station'] = [0]*len(df_sel)

        df_sel['pressure'] = df_sel['press']
        df_sel['time'] = df_sel['reltime']

        df_sel = df_sel.rename(columns={'reltime': 'datum', 'temp':'ta', 'latitude':'lat', 'longitude':'lon'})

        df_mi = df_sel.set_index(['station', 'time', 'pressure', 'hour'])

        df_mi = df_mi[~df_mi.index.duplicated(keep='first')]

        da = df_mi.to_xarray()

        compression =  {}
        for vars in da.keys():
            compression[vars] = {"compression": "gzip", "compression_opts": 5}

        da.to_netcdf('/users/staff/uvoggenberger/scratch/RHARM_2/'+file.split('/')[-1].split('.')[0]+'.nc', engine='h5netcdf', encoding=compression)

        return 0
    except:
        return 1, file


In [38]:
@ray.remote
def write_to_nc(file):
    try:
        df = pd.read_csv(file, low_memory=False)
        renamer = {}
        for i in df.keys():
            renamer[i] = i.split(' ')[-1]
        df = df.rename(columns=renamer)
        df_sel = df[['rh', 'u', 'v', 'press', 'temp', 'reltime', 'latitude', 'longitude']].copy()
        df_sel['reltime'] = pd.to_datetime(df_sel['reltime'], format='%Y-%m-%d%H:%M:%S+%f')


        splevs=[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
        df_sel = df_sel.copy()[np.isin(df_sel['press'], splevs)]

        avail = list(df_sel.press.drop_duplicates())
        for i in splevs:
            if i in avail:
                pass
            else:
                # print(i)
                # df_sel.loc[len(df_sel.index)] = [np.nan, np.nan, np.nan, i, np.nan, df_sel.reltime.iloc[0], df_sel.latitude.iloc[0], df_sel.longitude.iloc[0]]
                df_add = pd.DataFrame({"rh":np.nan, "u": np.nan, "v": np.nan, "press": i, "temp":np.nan, "reltime": df_sel.reltime.iloc[0], "latitude": df_sel.latitude.iloc[0], "longitude": df_sel.longitude.iloc[0]},
                   index=["1"])
                df_sel = pd.concat([df_sel, df_add], ignore_index=True)

        df_sel =  df_sel.copy()
        df_sel['hour'] = df_sel['reltime'].dt.hour
        hour_binary = np.array([0]*len(df_sel['hour']))
        hour_binary[np.logical_and(df_sel['hour'] > 6 , df_sel['hour'] <= 18)] = 1
        df_sel['hour'] = hour_binary


        df_sel['station'] = [0]*len(df_sel)

        df_sel['pressure'] = df_sel['press']

        sec_dates = []
        for i in df_sel.reltime:
            if i.hour > 18:
                sec_dates.append(int((i.ceil('1d') - pd.Timestamp('1900-01-01 00:00:00')).total_seconds()))
            else:
                sec_dates.append(int((i.floor('1d') - pd.Timestamp('1900-01-01 00:00:00')).total_seconds()))
        df_sel['reltime'] = sec_dates
        df_sel['time'] = df_sel['reltime']

        df_sel = df_sel.rename(columns={'reltime': 'datum', 'temp':'ta', 'latitude':'lat', 'longitude':'lon'})

        df_mi = df_sel.set_index(['time', 'pressure', 'hour'])

        df_mi = df_mi[~df_mi.index.duplicated(keep='first')]

        da = df_mi.to_xarray()

        # compression =  {}
        # for vars in da.keys():
        #     compression[vars] = {"compression": "gzip", "compression_opts": 5}

        # da.to_netcdf('/users/staff/uvoggenberger/scratch/RHARM_2/'+file.split('/')[-1].split('.')[0]+'.nc', engine='h5netcdf', encoding=compression)

        fn = '/users/staff/uvoggenberger/scratch/RHARM_2/'+file.split('/')[-1].split('.')[0]+'.nc'
        statid = file.split('/')[-1].split('.csv')[0]

        with nc.Dataset(fn, 'w', format='NETCDF4') as ds:

            # Dimensions:
            station = ds.createDimension('station', 1)
            # numdat = ds.createDimension('numdat', 1)
            time = ds.createDimension('time', len(df_sel.datum.drop_duplicates()))
            pressure = ds.createDimension('pressure', 16)
            hour = ds.createDimension('hour', 2)

            # Variables:
            lat = ds.createVariable('lat', 'f4', ('station',))
            lon = ds.createVariable('lon', 'f4', ('station',))
            press = ds.createVariable('press', 'f4', ('pressure',))
            ta = ds.createVariable('ta', 'f4', ('hour', 'pressure', 'time'))
            rh = ds.createVariable('rh', 'f4', ('hour', 'pressure', 'time'))
            v = ds.createVariable('v', 'f4', ('hour', 'pressure', 'time'))
            u = ds.createVariable('u', 'f4', ('hour', 'pressure', 'time'))
            datum = ds.createVariable('datum', 'f4', ('hour', 'time',))

            # Attributes:

            lat.long_name = "station latitude"
            lat.units = "degrees_north"
            lat.axis = "Y"
            lat.valid_range = [-90.,90.]
            lat.missing_value = -999.

            lon.long_name = "station longitude"
            lon.units = "degrees_east"
            lon.axis = "X"
            lon.valid_range = [-180., 180.]
            lon.missing_value = -999.

            press.long_name = "pressure"
            press.units = ""
            press.axis = "Z"
            press.valid_range = [1000.,100000.]
            press.missing_value = -999.

            datum.long_name = "datum"
            datum.units = "days since 1900-01-01 0:0:0"
            datum.axis = "T"
            datum.calendar = "gregorian"
            datum.missing_value = -999.

            ta.long_name = "temperature"
            ta.units = "K"
            ta.missing_value = -999.
            ta.valid_range = [0., 400.]
            ta.cell_methods = "time"

            rh.long_name = "relative_humidity"
            ta.units = "K"
            ta.missing_value = -999.
            ta.valid_range = [0., 400.]
            ta.cell_methods = "time"

            u.long_name = "eastward_wind"
            u.units = "m/s"
            u.missing_value = -999.
            u.valid_range = [0., 400.]
            u.cell_methods = "time"

            v.long_name = "northward_wind"
            v.units = "m/s"
            v.missing_value = -999.
            v.valid_range = [0., 400.]
            v.cell_methods = "time"

            # Global Attributes:
            ds.Conventions = "CF-1.1" ;
            ds.title = "" ;
            ds.institution = "University of Vienna" ;
            ds.history = "03/07/23" ;
            ds.source = "RHARM" ;
            ds.references = "www.univie.ac.at/theoret-met/research/raobcore" ;
            ds.Stationname = str(statid) ;


            # populating Variables:

            datum[:,:] =  np.transpose(np.array(da.datum.mean(dim='pressure', skipna=True)))
            pressure = splevs

            ta[:,:,:] =  np.transpose(np.array(da.ta[:,:,:]))
            rh[:,:,:] =  np.transpose(np.array(da.rh[:,:,:]))
            u[:,:,:] =  np.transpose(np.array(da.u[:,:,:]))
            v[:,:,:] =  np.transpose(np.array(da.v[:,:,:]))
            press[:] = splevs

            lat[:] = df_sel.lat.iloc[0]
            lon[:] = df_sel.lon.iloc[0]
            
        return 0
    except:
        return 1, file



In [39]:
result_ids = []
for file in glob.glob('/users/staff/uvoggenberger/scratch/RHARM_2/*.csv')[:]:
    result_ids.append(write_to_nc.remote(file))

results = ray.get(result_ids)
ray.shutdown()
results

2023-07-03 11:43:53,487	INFO worker.py:1528 -- Started a local Ray instance.


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [37]:
file = '/users/staff/uvoggenberger/scratch/RHARM_2/AFM00040948.csv'
renamer = {}
for i in df.keys():
    renamer[i] = i.split(' ')[-1]
df = df.rename(columns=renamer)
df_sel = df[['rh', 'u', 'v', 'press', 'temp', 'reltime', 'latitude', 'longitude']].copy()
df_sel['reltime'] = pd.to_datetime(df_sel['reltime'], format='%Y-%m-%d%H:%M:%S+%f')


splevs=[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
df_sel = df_sel.copy()[np.isin(df_sel['press'], splevs)]

avail = list(df_sel.press.drop_duplicates())
print(len(df_sel), len(df_sel.press.drop_duplicates()))        

for i in splevs:
    if i in avail:
        pass
    else:
        df_add = pd.DataFrame({"rh":np.nan, "u": np.nan, "v": np.nan, "press": i, "temp":np.nan, "reltime": df_sel.reltime.iloc[0], "latitude": df_sel.latitude.iloc[0], "longitude": df_sel.longitude.iloc[0]},
                   index=["1"])
        df_sel = pd.concat([df_sel, df_add], ignore_index=True)
        # df_sel.loc[len(df_sel.index)+1] = [np.nan, np.nan, np.nan, i, np.nan, df_sel.reltime.iloc[0], df_sel.latitude.iloc[0], df_sel.longitude.iloc[0]]
        # display(df_sel[df_sel.press == i])
        print(len(df_sel.index),len(df_sel.press.drop_duplicates()))        


print(len(df_sel.press.drop_duplicates()))  
print(df_sel.press.drop_duplicates())     
        
df_sel =  df_sel.copy()
df_sel['hour'] = df_sel['reltime'].dt.hour
hour_binary = np.array([0]*len(df_sel['hour']))
hour_binary[np.logical_and(df_sel['hour'] > 6 , df_sel['hour'] <= 18)] = 1
df_sel['hour'] = hour_binary


df_sel['station'] = [0]*len(df_sel)

df_sel['pressure'] = df_sel['press']


sec_dates = []
for i in df_sel.reltime:
    if i.hour > 18:
        sec_dates.append(int((i.ceil('1d') - pd.Timestamp('1900-01-01 00:00:00')).total_seconds()))
    else:
        sec_dates.append(int((i.floor('1d') - pd.Timestamp('1900-01-01 00:00:00')).total_seconds()))
df_sel['reltime'] = sec_dates
df_sel['time'] = df_sel['reltime']

df_sel = df_sel.rename(columns={'reltime': 'datum', 'temp':'ta', 'latitude':'lat', 'longitude':'lon'})

df_mi = df_sel.set_index(['time', 'pressure', 'hour'])

df_mi = df_mi[~df_mi.index.duplicated(keep='first')]

da = df_mi.to_xarray()

# compression =  {}
# for vars in da.keys():
#     compression[vars] = {"compression": "gzip", "compression_opts": 5}

# da.to_netcdf('/users/staff/uvoggenberger/scratch/RHARM_2/'+file.split('/')[-1].split('.')[0]+'.nc', engine='h5netcdf', encoding=compression)

fn = '/users/staff/uvoggenberger/scratch/RHARM_2/'+file.split('/')[-1].split('.')[0]+'.nc'
statid = file.split('/')[-1].split('.csv')[0]

with nc.Dataset(fn, 'w', format='NETCDF4') as ds:

    # Dimensions:
    station = ds.createDimension('station', 1)
    # numdat = ds.createDimension('numdat', 1)
    time = ds.createDimension('time', len(df_sel.datum.drop_duplicates()))
    pressure = ds.createDimension('pressure', 16)
    hour = ds.createDimension('hour', 2)

    # Variables:
    lat = ds.createVariable('lat', 'f4', ('station',))
    lon = ds.createVariable('lon', 'f4', ('station',))
    press = ds.createVariable('press', 'f4', ('pressure',))
    ta = ds.createVariable('ta', 'f4', ('hour', 'pressure', 'time'))
    rh = ds.createVariable('rh', 'f4', ('hour', 'pressure', 'time'))
    v = ds.createVariable('v', 'f4', ('hour', 'pressure', 'time'))
    u = ds.createVariable('u', 'f4', ('hour', 'pressure', 'time'))
    datum = ds.createVariable('datum', 'f4', ('hour', 'time',))

    # Attributes:

    lat.long_name = "station latitude"
    lat.units = "degrees_north"
    lat.axis = "Y"
    lat.valid_range = [-90.,90.]
    lat.missing_value = -999.

    lon.long_name = "station longitude"
    lon.units = "degrees_east"
    lon.axis = "X"
    lon.valid_range = [-180., 180.]
    lon.missing_value = -999.

    press.long_name = "pressure"
    press.units = ""
    press.axis = "Z"
    press.valid_range = [1000.,100000.]
    press.missing_value = -999.

    datum.long_name = "datum"
    datum.units = "days since 1900-01-01 0:0:0"
    datum.axis = "T"
    datum.calendar = "gregorian"
    datum.missing_value = -999.

    ta.long_name = "temperature"
    ta.units = "K"
    ta.missing_value = -999.
    ta.valid_range = [0., 400.]
    ta.cell_methods = "time"

    rh.long_name = "relative_humidity"
    ta.units = "K"
    ta.missing_value = -999.
    ta.valid_range = [0., 400.]
    ta.cell_methods = "time"

    u.long_name = "eastward_wind"
    u.units = "m/s"
    u.missing_value = -999.
    u.valid_range = [0., 400.]
    u.cell_methods = "time"

    v.long_name = "northward_wind"
    v.units = "m/s"
    v.missing_value = -999.
    v.valid_range = [0., 400.]
    v.cell_methods = "time"

    # Global Attributes:
    ds.Conventions = "CF-1.1" ;
    ds.title = "" ;
    ds.institution = "University of Vienna" ;
    ds.history = "03/07/23" ;
    ds.source = "RHARM" ;
    ds.references = "www.univie.ac.at/theoret-met/research/raobcore" ;
    ds.Stationname = str(statid) ;


    # populating Variables:

    datum[:,:] =  np.transpose(np.array(da.datum.mean(dim='pressure', skipna=True)))
    pressure = splevs

    ta[:,:,:] =  np.transpose(np.array(da.ta[:,:,:]))
    rh[:,:,:] =  np.transpose(np.array(da.rh[:,:,:]))
    u[:,:,:] =  np.transpose(np.array(da.u[:,:,:]))
    v[:,:,:] =  np.transpose(np.array(da.v[:,:,:]))
    press[:] = splevs

    lat[:] = df_sel.lat.iloc[0]
    lon[:] = df_sel.lon.iloc[0]

157528 13
157529 14
157530 15
157531 16
16
0          50000
1          40000
2          30000
3          20000
4          10000
5          70000
9          15000
541        25000
592         7000
613         5000
614         3000
615         2000
750         1000
157528     85000
157529     92500
157530    100000
Name: press, dtype: int64


In [10]:
len(df_sel.press.drop_duplicates())

15

## testing

In [71]:
glob.glob('/users/staff/uvoggenberger/scratch/RHARM_2/*.csv')[0].split('/')[-1].split('.')[0]

'AFM00040948'

In [52]:
df = pd.read_csv('./test.csv', low_memory=False)
renamer = {}
for i in df.keys():
    renamer[i] = i.split(' ')[-1]
print(renamer)
df = df.rename(columns=renamer)
df_sel = df[['rh', 'u', 'v', 'press', 'temp', 'reltime', 'latitude', 'longitude']].copy()
df_sel['reltime'] = pd.to_datetime(df_sel['reltime'], format='%Y-%m-%d%H:%M:%S+%f')
df_sel

{'id': 'id', ' header_id': 'header_id', ' time': 'time', ' press': 'press', ' geopot': 'geopot', ' lvltyp1': 'lvltyp1', ' lvltyp2': 'lvltyp2', ' pflag': 'pflag', ' zflag': 'zflag', ' tflag': 'tflag', ' dpdp': 'dpdp', ' fp': 'fp', ' asc_': 'asc_', ' sza': 'sza', ' temp': 'temp', ' temp_product': 'temp_product', ' temp_product_cor_temp': 'temp_product_cor_temp', ' temp_product_u_cor_temp': 'temp_product_u_cor_temp', ' temp_product_cor_temp_tl': 'temp_product_cor_temp_tl', ' temp_product_u_cor_temp_tl': 'temp_product_u_cor_temp_tl', ' temp_product_cor_intercomparison_temp': 'temp_product_cor_intercomparison_temp', ' temp_product_u_cor_intercomparison_temp': 'temp_product_u_cor_intercomparison_temp', ' temp_h': 'temp_h', ' err_temp_h': 'err_temp_h', ' rh': 'rh', ' rh_product': 'rh_product', ' rh_product_cor_rh': 'rh_product_cor_rh', ' rh_product_u_cor_rh': 'rh_product_u_cor_rh', ' rh_product_cor_rh_tl': 'rh_product_cor_rh_tl', ' rh_product_u_cor_rh_tl': 'rh_product_u_cor_rh_tl', ' rh_produ

,rh,u,v,press,temp,reltime,latitude,longitude
0,14.00,8.66,5.00,50000,255.15,1966-01-07 00:00:00,34.55,69.2167
1,29.00,7.00,12.12,40000,242.15,1966-01-07 00:00:00,34.55,69.2167
2,16.00,NaN,NaN,30000,226.95,1966-01-07 00:00:00,34.55,69.2167
3,12.00,33.77,-19.50,20000,214.35,1966-01-07 00:00:00,34.55,69.2167
4,10.00,NaN,NaN,10000,205.35,1966-01-07 00:00:00,34.55,69.2167
...,...,...,...,...,...,...,...,...
443042,9.96,NaN,NaN,9020,201.65,2021-08-13 23:07:00,34.55,69.2167
443043,12.61,NaN,NaN,7780,197.45,2021-08-13 23:07:00,34.55,69.2167
443044,5.26,0.81,-9.26,7000,203.05,2021-08-13 23:07:00,34.55,69.2167
443045,5.30,NaN,NaN,6940,203.25,2021-08-13 23:07:00,34.55,69.2167


In [53]:
splevs=[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
len(splevs)
df_sel = df_sel.copy()[np.isin(df_sel['press'], splevs)]
# df_sel.press.drop_duplicates()
df_sel

,rh,u,v,press,temp,reltime,latitude,longitude
0,14.00,8.66,5.00,50000,255.15,1966-01-07 00:00:00,34.55,69.2167
1,29.00,7.00,12.12,40000,242.15,1966-01-07 00:00:00,34.55,69.2167
2,16.00,NaN,NaN,30000,226.95,1966-01-07 00:00:00,34.55,69.2167
3,12.00,33.77,-19.50,20000,214.35,1966-01-07 00:00:00,34.55,69.2167
4,10.00,NaN,NaN,10000,205.35,1966-01-07 00:00:00,34.55,69.2167
...,...,...,...,...,...,...,...,...
443035,NaN,30.90,0.00,25000,239.85,2021-08-13 23:07:00,34.55,69.2167
443036,NaN,31.28,-2.74,20000,228.45,2021-08-13 23:07:00,34.55,69.2167
443037,2.92,26.39,4.65,15000,214.65,2021-08-13 23:07:00,34.55,69.2167
443041,13.26,23.70,0.00,10000,199.65,2021-08-13 23:07:00,34.55,69.2167


In [54]:
avail = list(df_sel.press.drop_duplicates())
for i in splevs:
    if i in avail:
        pass
    else:
        print(i)
        df_sel.loc[len(df_sel.index)] = [np.nan, np.nan, np.nan, i, np.nan, df_sel.reltime.iloc[0], df_sel.latitude.iloc[0], df_sel.longitude.iloc[0]]
        
df_sel

85000
92500
100000


,rh,u,v,press,temp,reltime,latitude,longitude
0,14.00,8.66,5.00,50000,255.15,1966-01-07 00:00:00,34.55,69.2167
1,29.00,7.00,12.12,40000,242.15,1966-01-07 00:00:00,34.55,69.2167
2,16.00,NaN,NaN,30000,226.95,1966-01-07 00:00:00,34.55,69.2167
3,12.00,33.77,-19.50,20000,214.35,1966-01-07 00:00:00,34.55,69.2167
4,10.00,NaN,NaN,10000,205.35,1966-01-07 00:00:00,34.55,69.2167
...,...,...,...,...,...,...,...,...
443037,2.92,26.39,4.65,15000,214.65,2021-08-13 23:07:00,34.55,69.2167
443041,13.26,23.70,0.00,10000,199.65,2021-08-13 23:07:00,34.55,69.2167
443044,5.26,0.81,-9.26,7000,203.05,2021-08-13 23:07:00,34.55,69.2167
157528,NaN,NaN,NaN,85000,NaN,1966-01-07 00:00:00,34.55,69.2167


In [57]:
df_sel =  df_sel.copy()
df_sel['hour'] = df_sel['reltime'].dt.hour
hour_binary = np.array([0]*len(df_sel['hour']))
hour_binary[np.logical_and(df_sel['hour'] > 6 , df_sel['hour'] <= 18)] = 1
df_sel['hour'] = hour_binary
df_sel

,rh,u,v,press,temp,reltime,latitude,longitude,hour
0,14.00,8.66,5.00,50000,255.15,1966-01-07 00:00:00,34.55,69.2167,0
1,29.00,7.00,12.12,40000,242.15,1966-01-07 00:00:00,34.55,69.2167,0
2,16.00,NaN,NaN,30000,226.95,1966-01-07 00:00:00,34.55,69.2167,0
3,12.00,33.77,-19.50,20000,214.35,1966-01-07 00:00:00,34.55,69.2167,0
4,10.00,NaN,NaN,10000,205.35,1966-01-07 00:00:00,34.55,69.2167,0
...,...,...,...,...,...,...,...,...,...
443037,2.92,26.39,4.65,15000,214.65,2021-08-13 23:07:00,34.55,69.2167,0
443041,13.26,23.70,0.00,10000,199.65,2021-08-13 23:07:00,34.55,69.2167,0
443044,5.26,0.81,-9.26,7000,203.05,2021-08-13 23:07:00,34.55,69.2167,0
157528,NaN,NaN,NaN,85000,NaN,1966-01-07 00:00:00,34.55,69.2167,0


In [58]:
df_sel['station'] = ['1']*len(df_sel)
df_sel

,rh,u,v,press,temp,reltime,latitude,longitude,hour,station
0,14.00,8.66,5.00,50000,255.15,1966-01-07 00:00:00,34.55,69.2167,0,1
1,29.00,7.00,12.12,40000,242.15,1966-01-07 00:00:00,34.55,69.2167,0,1
2,16.00,NaN,NaN,30000,226.95,1966-01-07 00:00:00,34.55,69.2167,0,1
3,12.00,33.77,-19.50,20000,214.35,1966-01-07 00:00:00,34.55,69.2167,0,1
4,10.00,NaN,NaN,10000,205.35,1966-01-07 00:00:00,34.55,69.2167,0,1
...,...,...,...,...,...,...,...,...,...,...
443037,2.92,26.39,4.65,15000,214.65,2021-08-13 23:07:00,34.55,69.2167,0,1
443041,13.26,23.70,0.00,10000,199.65,2021-08-13 23:07:00,34.55,69.2167,0,1
443044,5.26,0.81,-9.26,7000,203.05,2021-08-13 23:07:00,34.55,69.2167,0,1
157528,NaN,NaN,NaN,85000,NaN,1966-01-07 00:00:00,34.55,69.2167,0,1


In [59]:
df_sel['pressure'] = df_sel['press']
df_sel['time'] = df_sel['reltime']
df_sel

,rh,u,v,press,temp,reltime,latitude,longitude,hour,station,pressure,time
0,14.00,8.66,5.00,50000,255.15,1966-01-07 00:00:00,34.55,69.2167,0,1,50000,1966-01-07 00:00:00
1,29.00,7.00,12.12,40000,242.15,1966-01-07 00:00:00,34.55,69.2167,0,1,40000,1966-01-07 00:00:00
2,16.00,NaN,NaN,30000,226.95,1966-01-07 00:00:00,34.55,69.2167,0,1,30000,1966-01-07 00:00:00
3,12.00,33.77,-19.50,20000,214.35,1966-01-07 00:00:00,34.55,69.2167,0,1,20000,1966-01-07 00:00:00
4,10.00,NaN,NaN,10000,205.35,1966-01-07 00:00:00,34.55,69.2167,0,1,10000,1966-01-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
443037,2.92,26.39,4.65,15000,214.65,2021-08-13 23:07:00,34.55,69.2167,0,1,15000,2021-08-13 23:07:00
443041,13.26,23.70,0.00,10000,199.65,2021-08-13 23:07:00,34.55,69.2167,0,1,10000,2021-08-13 23:07:00
443044,5.26,0.81,-9.26,7000,203.05,2021-08-13 23:07:00,34.55,69.2167,0,1,7000,2021-08-13 23:07:00
157528,NaN,NaN,NaN,85000,NaN,1966-01-07 00:00:00,34.55,69.2167,0,1,85000,1966-01-07 00:00:00


In [60]:
df_sel = df_sel.rename(columns={'reltime': 'datum', 'temp':'ta', 'latitude':'lat', 'longitude':'lon'})

In [61]:
df_mi = df_sel.set_index(['station', 'time', 'pressure', 'hour'])
df_mi

rh      u      v  press      ta  \
station time                pressure hour                                       
1       1966-01-07 00:00:00 50000    0     14.00   8.66   5.00  50000  255.15   
                            40000    0     29.00   7.00  12.12  40000  242.15   
                            30000    0     16.00    NaN    NaN  30000  226.95   
                            20000    0     12.00  33.77 -19.50  20000  214.35   
                            10000    0     10.00    NaN    NaN  10000  205.35   
...                                          ...    ...    ...    ...     ...   
        2021-08-13 23:07:00 15000    0      2.92  26.39   4.65  15000  214.65   
                            10000    0     13.26  23.70   0.00  10000  199.65   
                            7000     0      5.26   0.81  -9.26   7000  203.05   
        1966-01-07 00:00:00 85000    0       NaN    NaN    NaN  85000     NaN   
                            92500    0       NaN    NaN    NaN  92500     NaN   

                                                        datum    lat      lon  
station time                pressure hour                                      
1       1966-01-07 00:00:00 50000    0    1966-01-07 00:00:00  34.55  69.2167  
                            40000    0    1966-01-07 00:00:00  34.55  69.2167  
                            30000    0    1966-01-07 00:00:00  34.55  69.2167  
                            20000    0    1966-01-07 00:00:00  34.55  69.2167  
                            10000    0    1966-01-07 00:00:00  34.55  69.2167  
...                                                       ...    ...      ...  
        2021-08-13 23:07:00 15000    0    2021-08-13 23:07:00  34.55  69.2167  
                            10000    0    2021-08-13 23:07:00  34.55  69.2167  
                            7000     0    2021-08-13 23:07:00  34.55  69.2167  
        1966-01-07 00:00:00 85000    0    1966-01-07 00:00:00  34.55  69.2167  
                            92500    0    1966-01-07 00:00:00  34.55  69.2167  

[157530 rows x 8 columns]

In [62]:
df_mi = df_mi[~df_mi.index.duplicated(keep='first')]
df_mi

rh      u      v  press      ta  \
station time                pressure hour                                       
1       1966-01-07 00:00:00 50000    0     14.00   8.66   5.00  50000  255.15   
                            40000    0     29.00   7.00  12.12  40000  242.15   
                            30000    0     16.00    NaN    NaN  30000  226.95   
                            20000    0     12.00  33.77 -19.50  20000  214.35   
                            10000    0     10.00    NaN    NaN  10000  205.35   
...                                          ...    ...    ...    ...     ...   
        2021-08-13 23:07:00 15000    0      2.92  26.39   4.65  15000  214.65   
                            10000    0     13.26  23.70   0.00  10000  199.65   
                            7000     0      5.26   0.81  -9.26   7000  203.05   
        1966-01-07 00:00:00 85000    0       NaN    NaN    NaN  85000     NaN   
                            92500    0       NaN    NaN    NaN  92500     NaN   

                                                        datum    lat      lon  
station time                pressure hour                                      
1       1966-01-07 00:00:00 50000    0    1966-01-07 00:00:00  34.55  69.2167  
                            40000    0    1966-01-07 00:00:00  34.55  69.2167  
                            30000    0    1966-01-07 00:00:00  34.55  69.2167  
                            20000    0    1966-01-07 00:00:00  34.55  69.2167  
                            10000    0    1966-01-07 00:00:00  34.55  69.2167  
...                                                       ...    ...      ...  
        2021-08-13 23:07:00 15000    0    2021-08-13 23:07:00  34.55  69.2167  
                            10000    0    2021-08-13 23:07:00  34.55  69.2167  
                            7000     0    2021-08-13 23:07:00  34.55  69.2167  
        1966-01-07 00:00:00 85000    0    1966-01-07 00:00:00  34.55  69.2167  
                            92500    0    1966-01-07 00:00:00  34.55  69.2167  

[157520 rows x 8 columns]

In [63]:
df_mi.to_xarray()

<xarray.Dataset>
Dimensions:   (station: 1, time: 17284, pressure: 16, hour: 2)
Coordinates:
  * station   (station) object '1'
  * time      (time) datetime64[ns] 1966-01-07 ... 2021-08-13T23:07:00
  * pressure  (pressure) int64 1000 2000 3000 5000 ... 70000 85000 92500 100000
  * hour      (hour) int64 0 1
Data variables:
    rh        (station, time, pressure, hour) float64 nan nan nan ... nan nan
    u         (station, time, pressure, hour) float64 nan nan nan ... nan nan
    v         (station, time, pressure, hour) float64 nan nan nan ... nan nan
    press     (station, time, pressure, hour) float64 nan nan nan ... nan nan
    ta        (station, time, pressure, hour) float64 nan nan nan ... nan nan
    datum     (station, time, pressure, hour) datetime64[ns] NaT NaT ... NaT NaT
    lat       (station, time, pressure, hour) float64 nan nan nan ... nan nan
    lon       (station, time, pressure, hour) float64 nan nan nan ... nan nan

In [69]:
xr.open_dataset('/users/staff/leo/fastscratch/rise/1.0/exp01/070414/rharm_h_070414.nc',)

<xarray.Dataset>
Dimensions:  (station: 1, pressure: 16, hour: 2, time: 15443)
Dimensions without coordinates: station, pressure, hour, time
Data variables:
    lat      (station) float32 ...
    lon      (station) float32 ...
    press    (pressure) float32 ...
    ta       (hour, pressure, time) float32 ...
    rh       (hour, pressure, time) float32 ...
    v        (hour, pressure, time) float32 ...
    u        (hour, pressure, time) float32 ...
    datum    (hour, time) datetime64[ns] ...
Attributes:
    Conventions:  CF-1.1
    title:        
    institution:  University of Vienna
    history:      17/09/20
    source:       IGRA_H
    Stationname:  USM00070414